In [38]:
from importlib import reload
from time import sleep

import numpy as np
import polars as pl
from plotly import express as px
from plotly import io as pio

import cool_search as cool

plot_temp = pio.templates["plotly_dark"]
plot_temp.layout.width = 400
plot_temp.layout.height = 300
plot_temp.layout.autosize = False
pio.templates.default = plot_temp


## 1D

In [39]:
def f(x):
    """1D curve, with a clear minimum
    - slightly slow
    """
    sleep(0.1)
    return np.sqrt((x - 2) ** 2 + 1) + np.sin(x / 2)


X = np.linspace(-10, 10, 300)
Y = f(X)
px.line(x=X, y=Y)


In [40]:
reload(cool)
search = cool.CoolSearch(
    f,
    {"x": (-10, 10)},
)
display(search.samples)
print(search)

print("example grid:")
display(search.get_grid(5))
search.grid_search(20)

px.scatter(
    search.samples,
    x="x",
    y="score",
    color="runtime",
    title="current samples",
)

x,score,runtime
f32,f64,f64


1 dimensional search
  - has 0 samples
example grid:


x
f32
-10.0
-5.0
0.0
5.0
10.0


searching 20 new parameter points
total runtime: 2.0134 s + overhead: 0.0020 s.


# 2D

In [49]:
def g(x, y):
    """2D function, with a clear minimum.
    - slightly slow"""
    sleep(0.1)
    return np.sqrt((((x+1)**2 + y**2) - 2) ** 2 + 1) + 3 * np.sin((y) ** 2 / 2)


search = cool.CoolSearch(
    g,
    dict.fromkeys(["x", "y"], (-10, 10)),
)

grid = search.get_grid(20)


px.scatter(
    grid,
    x="x",
    y="y",
    color=g(grid[:, 0], grid[:, 1]),
    title=f"GT evaluatead on {len(grid)} points"
)


shape: (400,)
Series: 'x' [f32]
[
	-10.0
	-10.0
	-10.0
	-10.0
	-10.0
	…
	10.0
	10.0
	10.0
	10.0
	10.0
]


In [44]:
search.grid_search(5)
min_points = search.samples.filter(pl.col("score") == pl.col("score").min())
print(f"mean of {len(min_points)} minimum points:")
# TODO test target runtimes

px.scatter(
    search.samples,
    x="x",
    y="y",
    color="score",
    size="runtime",
    title=f"grid search of {len(search.samples)} points"
)


searching 25 new parameter points
total runtime: 2.5099 s + overhead: 0.0021 s.
mean of 1 minimum points:


In [53]:
reload(cool)
search = cool.CoolSearch(
    g,
    {
        "x": (-10, 10),
        "y": (-5, 5),
    },
    {
        "x": "float",
        "y": "int",
    },
)
print(search)

search.get_grid(50)

2 dimensional search
  - has 0 samples


x,y
f32,i32
-10.0,-5
-10.0,-4
-10.0,-3
-10.0,-2
-10.0,-1
…,…
10.0,1
10.0,2
10.0,3
